# KITTI dataset parser

## Extract function

In [2]:
import numpy as np
import pykitti
from source import parseTrackletXML as xmlParser



def load_dataset(date, drive, calibrated=False, frame_range=None):
    """
    Loads the dataset with `date` and `drive`.
    
    Parameters
    ----------
    date        : Dataset creation date.
    drive       : Dataset drive.
    calibrated  : Flag indicating if we need to parse calibration data. Defaults to `False`.
    frame_range : Range of frames. Defaults to `None`.

    Returns
    -------
    Loaded dataset of type `raw`.
    """
    dataset = pykitti.raw(basedir, date, drive)

    # Load the data
    if calibrated:
        dataset.load_calib()  # Calibration data are accessible as named tuples

    np.set_printoptions(precision=4, suppress=True)
    print('\nDrive: ' + str(dataset.drive))
    print('\nFrame range: ' + str(dataset.frames))

    if calibrated:
        print('\nIMU-to-Velodyne transformation:\n' + str(dataset.calib.T_velo_imu))
        print('\nGray stereo pair baseline [m]: ' + str(dataset.calib.b_gray))
        print('\nRGB stereo pair baseline [m]: ' + str(dataset.calib.b_rgb))

    return dataset



def kitti_tracklets_extraction(basedir, date, drive, calibrated=False, frame_range=None):
    """
    input:
        dataset path
            basedir, date, drive
        
    output:
        tracklets: list of tracklet
        tracklet['pc']
        tracklet['rect']
        tracklet['trans']
        tracklet['rot]
    
    """
    dataset = load_dataset(date, drive, calibrated=False, frame_range=None)
    dataset_velo = list(dataset.velo)
    tracklets = xmlParser.parseXML('../dataset/{}/{}_drive_{}_sync/tracklet_labels.xml'.format(date, date, drive) )
    
    # loop over tracklets
    parse_tracklets = []
    
    for i, tracklet in enumerate(tracklets):
        
        
        parse_tracklet = {}
        
        parse_tracklet['pc'] = []
        parse_tracklet['rect'] = []
        parse_tracklet['trans'] = []
        parse_tracklet['rot'] = []
        parse_tracklet['type'] = []

        # this part is inspired by kitti object development kit matlab code: computeBox3D
        h, w, l = tracklet.size
        # in velodyne coordinates around zero point and without orientation yet
        trackletBox = np.array([
            [-l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2],
            [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
            [0.0, 0.0, 0.0, 0.0, h, h, h, h]
        ])
        # loop over all data in tracklet
        for translation, rotation, state, occlusion, truncation, amtOcclusion, amtBorders, absoluteFrameNumber in tracklet:
            
            # determine if object is in the image; otherwise continue
            if truncation not in (xmlParser.TRUNC_IN_IMAGE, xmlParser.TRUNC_TRUNCATED):
                continue
            
            # re-create 3D bounding box in velodyne coordinate system
            yaw = rotation[2]  # other rotations are supposedly 0
            assert np.abs(rotation[:2]).sum() == 0, 'object rotations other than yaw given!'
            rotMat = np.array([
                [np.cos(yaw), -np.sin(yaw), 0.0],
                [np.sin(yaw), np.cos(yaw), 0.0],
                [0.0, 0.0, 1.0]
            ])
            cornerPosInVelo = np.dot(rotMat, trackletBox) + np.tile(translation, (8, 1)).T
            
            # extract point cloud
            velo_pc = dataset_velo[absoluteFrameNumber].copy()
            min_val = np.min(cornerPosInVelo, axis = 1)
            max_val = np.max(cornerPosInVelo, axis = 1)
            
            for j in range(3):
                velo_pc = velo_pc[ np.logical_and( velo_pc[:,j] < max_val[j], velo_pc[:,j] > min_val[j] )]    
            
            parse_tracklet['type'].append(tracklet.objectType)
            parse_tracklet['rect'].append(cornerPosInVelo)
            parse_tracklet['trans'].append(translation)
            parse_tracklet['rot'].append(rotation)
            parse_tracklet['pc'].append(velo_pc)

        parse_tracklets.append(parse_tracklet)          
  
    return parse_tracklets
    
    


## Test

In [4]:
import os

basedir = os.path.abspath('../dataset')
date = '2011_09_26'
drive = '0001'

parse_tracklets = kitti_tracklets_extraction(basedir, date, drive)

print ( type(parse_tracklets) )


Drive: 2011_09_26_drive_0001_sync

Frame range: None
Parsing tracklet file ../dataset/2011_09_26/2011_09_26_drive_0001_sync/tracklet_labels.xml
File contains 15 tracklets
Loaded 15 tracklets.
<class 'list'>


### Visualization

In [9]:
from visualization import *


%matplotlib qt5

fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(1,1,1, projection='3d')

pc_all = []

for i in range( len( parse_tracklets ) ):
    
    parse_tracklet = parse_tracklets[i]
    
    for j in range( len( parse_tracklet['pc'] ) ):
                   
        pc = parse_tracklet['pc'][j]
        pc = resample_point_cloud(pc, 0.1)
        display_point_cloud_ax_color(ax, pc, color = plt.cm.RdYlBu(i / len( parse_tracklets ) ))
        
        pc_all.append(np.min(pc, axis = 0) )
        pc_all.append(np.max(pc, axis = 0) )

# make axis equal
points = np.array(pc_all)
p_min = np.min(points, axis=0)
p_max = np.max(points, axis=0)
max_range = np.array([p_max[0]-p_min[0], p_max[1]-p_min[1], p_max[2]-p_min[2]]).max() / 2.0

mid_x = (p_max[0]+p_min[0]) * 0.5
mid_y = (p_max[1]+p_min[1]) * 0.5
mid_z = (p_max[2]+p_min[2]) * 0.5
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

plt.show()


## Parsing all dataset

In [12]:
import os


"""
KITTI dataset
configuration
"""

basedir = os.path.abspath('../dataset')
date = '2011_09_26'

## parse drives
import re

p = re.compile(date + '_drive_\d{4}')

drives = []

for root,d_names,f_names in os.walk(root_path):
    for d in d_names:
        m = p.match(d)
        if m:
            print('Match found: ', m.group())
            drives.append(m.group().split('_')[-1])


"""
Parsing
"""

parsed_data = {}
parsed_data[date] = {}

for d in drives:
    parsed_data[date][d] = parse_tracklets = kitti_tracklets_extraction(basedir, date, d)


Match found:  2011_09_26_drive_0011
Match found:  2011_09_26_drive_0001
Match found:  2011_09_26_drive_0002
Match found:  2011_09_26_drive_0005
Match found:  2011_09_26_drive_0009
Match found:  2011_09_26_drive_0013
Match found:  2011_09_26_drive_0014
Match found:  2011_09_26_drive_0017

Drive: 2011_09_26_drive_0011_sync

Frame range: None
Parsing tracklet file ../dataset/2011_09_26/2011_09_26_drive_0011_sync/tracklet_labels.xml
File contains 20 tracklets
Loaded 20 tracklets.

Drive: 2011_09_26_drive_0001_sync

Frame range: None
Parsing tracklet file ../dataset/2011_09_26/2011_09_26_drive_0001_sync/tracklet_labels.xml
File contains 15 tracklets
Loaded 15 tracklets.

Drive: 2011_09_26_drive_0002_sync

Frame range: None
Parsing tracklet file ../dataset/2011_09_26/2011_09_26_drive_0002_sync/tracklet_labels.xml
File contains 3 tracklets
Loaded 3 tracklets.

Drive: 2011_09_26_drive_0005_sync

Frame range: None
Parsing tracklet file ../dataset/2011_09_26/2011_09_26_drive_0005_sync/tracklet_l

In [16]:
import pickle
## Save pickle
with open('parsed_kitti_data.pickle', 'wb') as f:
    pickle.dump(parsed_data, f, protocol=2)
    
print("\n\nSaved new dataset, protocol=2")



Saved new dataset, protocol=2
